Using data from Kevin Ummel and the American Community Survey Supplemental Poverty Measure research file.

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
import microdf as mdf
from ubicenter import format_fig
import pyreadr

cu = pd.read_csv("../data/carbon_footprint_2018.csv")
cuid_sporder = pd.DataFrame(
    list(pyreadr.read_r("../data/CUID-SPORDER.rds").values())[0]
)
p = pd.read_csv("../data/acs_poverty.csv.gz")


In [5]:
# Per Kevin Ummel, serialno is first 13 characters of CUID
def cuid2serialno(cuid):
    return cuid.str[6:13].astype(int)


cuid_sporder["serialno"] = cuid2serialno(cuid_sporder.CUID)
cu["serialno"] = cuid2serialno(cu.CUID)
# Household dataset formed from the CU dataset.
h_co2 = cu.drop(columns="CUID").groupby("serialno").sum().reset_index()
# Columns to rename as they're aggregated to household.
h_co2_cols = h_co2.columns.drop("serialno")
h_co2.rename(
    columns=dict(zip(h_co2_cols, [i + "_hh" for i in h_co2_cols])),
    inplace=True,
)
# Assign emissions equally across household members.
p_per_h = (
    p.groupby("serialno")
    .size()
    .reset_index()
    .rename(columns={0: "people_in_hh"})
)
p = p.merge(p_per_h, on="serialno").merge(ch, on="serialno")
for i in h_co2:
    p[i] = p[i + "_hh"] / p.people_in_hh

SPMU_COLS = ["id", "povthreshold", "resources"]
p["person"] = 1
s = (
    p.groupby(["spm_" + i for i in SPMU_COLS])[["person", "tco2"]]
    .sum()
    .reset_index()
)

total_co2 = mdf.weighted_sum(p, "tco2", "wt")
total_pop = p.wt.sum()
co2_pp = total_co2 / total_pop
# s["co2"] =
# s["dividend"] = s.credits * co2_per_credit * PRICE
# s["spm_resources_new"] = s.spm_resources + s.dividend - s.co2_tax
# # Merge back to person.
# p = p.merge(s[["spm_id", "spm_resources_new"]], on="spm_id")


In [31]:
p["age_group"] = np.where(p.age < 18, "Child", "Adult")

In [45]:
def carbon_dividend(price):
    def pov_metrics(p):
        poverty = mdf.poverty_rate(
            p, "spm_resources_new", "spm_povthreshold", "wt",
        )
        deep_poverty = mdf.deep_poverty_rate(
            p, "spm_resources_new", "spm_povthreshold", "wt",
        )
        return pd.Series(dict(poverty=poverty, deep_poverty=deep_poverty))

    dividend = co2_pp * price
    s["dividend"] = s.person * dividend
    s["co2_tax"] = s.tco2 * price
    s["spm_resources_new"] = s.spm_resources + s.dividend - s.co2_tax
    p2 = p.merge(s[["spm_id", "spm_resources_new"]], on="spm_id")
    pov_overall = pd.DataFrame(pov_metrics(p2)).T
    pov_overall["age_group"] = "All"
    pov_age = p2.groupby("age_group").apply(pov_metrics).reset_index()
    res = pd.concat([pov_overall, pov_age], axis=0)
    res["price"] = price
    return res


In [49]:
carbon_price = pd.concat(
    [carbon_dividend(i) for i in np.arange(0, 15, 5)]
).reset_index(drop=True)
base_poverty = carbon_price[carbon_price.price == 0].rename(
    columns={"poverty": "base_poverty", "deep_poverty": "base_deep_poverty"}
)
carbon_price = carbon_price.merge(base_poverty, on="age_group")
carbon_price["poverty_chg"] = (
    carbon_price.poverty / carbon_price.base_poverty - 1
)
carbon_price["deep_poverty_chg"] = (
    carbon_price.deep_poverty / carbon_price.base_deep_poverty - 1
)
carbon_price


In [18]:
fig = px.line(
    carbon_price,
    "price",
    "poverty_rate_chg",
    title="Carbon dividend poverty impact",
    labels=dict(price="Carbon price", poverty_rate_chg="Change to poverty rate"),
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat="%")
format_fig(fig)


In [8]:
mdf.poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.1532896866553634

In [6]:
p

,serialno,sporder,wt,age,spm_id,spm_povthreshold,spm_resources,people_in_hh,tco2_hh,tco2_direct_hh,invest_assets_hh,tco2,tco2_direct,invest_assets,credits,spm_resources_new
0,1.0,1.0,95.0,31.0,1.001000e+03,13257.998717,84922.922400,1,55.80,15.08,0.4840,55.800,15.080000,0.484000,1.0,84451.387791
1,2.0,1.0,64.0,69.0,2.001000e+03,15260.377969,98005.314955,2,68.60,23.55,1.0080,34.300,11.775000,0.504000,1.0,97707.245737
2,2.0,2.0,65.0,71.0,2.001000e+03,15260.377969,98005.314955,2,68.60,23.55,1.0080,34.300,11.775000,0.504000,1.0,97707.245737
3,3.0,1.0,19.0,58.0,3.001000e+03,15946.591880,41795.643698,2,30.23,16.52,0.7650,15.115,8.260000,0.382500,1.0,42073.124480
4,3.0,2.0,17.0,59.0,3.001000e+03,15946.591880,41795.643698,2,30.23,16.52,0.7650,15.115,8.260000,0.382500,1.0,42073.124480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061059,1400926.0,2.0,126.0,66.0,1.400926e+09,25706.194000,97274.729642,3,86.40,39.80,0.4790,28.800,13.266667,0.159667,1.0,97075.125815
3061060,1400926.0,3.0,112.0,42.0,1.400926e+09,25706.194000,97274.729642,3,86.40,39.80,0.4790,28.800,13.266667,0.159667,1.0,97075.125815
3061061,1400927.0,1.0,93.0,27.0,1.400927e+09,34619.753000,44792.800734,3,37.35,7.78,0.1124,12.450,2.593333,0.037467,1.0,45328.946907
3061062,1400927.0,2.0,69.0,56.0,1.400927e+09,34619.753000,44792.800734,3,37.35,7.78,0.1124,12.450,2.593333,0.037467,1.0,45328.946907


## Analysis

Total CO2

Compare to 6.6 billion tons CO2 equivalent ([EPA](https://www.epa.gov/ghgemissions/inventory-us-greenhouse-gas-emissions-and-sinks)), though includes other GHGs.

In [4]:
total_co2 / 1e9

6.884031485652741

In [5]:
mdf.poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.1532896866553634

In [6]:
mdf.poverty_rate(p, "spm_resources_new", "spm_povthreshold", "wt")

0.15018853668734483

In [10]:
1 - mdf.poverty_rate(p, "spm_resources_new", "spm_povthreshold", "wt") / mdf.poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.020230649795708655

In [11]:
1 - mdf.deep_poverty_rate(p, "spm_resources_new", "spm_povthreshold", "wt") / mdf.deep_poverty_rate(p, "spm_resources", "spm_povthreshold", "wt")

0.024789704725417305